# Zero to Hero

## 1.11 Arbol training/testing

El objetivo es calcular la ganancia en testing de un arbol de decision

In [ ]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
library( "caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("X:\\gdrive\\austral2023v\\")  #Aqui se debe poner la ruta de la PC local

In [ ]:
dataset <- fread("./datasets/dataset_pequeno.csv")   #cargo el dataset #cargo el dataset
dataset <- dataset[ foto_mes==202107 ]  # me quedo solo con el periodo 202107

In [ ]:
#creo training con 70%  y testing con 30%
set.seed(13)
train_rows <- createDataPartition(dataset$clase_ternaria, p= 0.70,list= FALSE)
dtrain  <- dataset[ train_rows]
dtest  <-  dataset[ -train_rows]

Entreno el modelo en los datos de training

In [ ]:
param  <- list("cp"= -0.5,
               "minsplit"=  900,
               "minbucket"= 440,
               "maxdepth"= 5
              )

Entreno el modelo en los datos de training

In [ ]:
#genero el modelo
modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                        data= dtrain,
                        xval= 0,
                        control= param)

aplico el modelo a los datos de testing

In [ ]:
prediccion  <- predict( modelo, dtest, type = "prob")

### Calculo la ganancia del modelo en los datos de testing

a partir de la prediccion, calculo la probabilidad de BAJA+2 de cada registro de testing  **dtest**

In [ ]:
prob_baja2  <- prediccion[, "BAJA+2"]

Para el cálculo de la ganancia del modelo aplicado a testing debo tener en cuenta lo siguiente
* Solo envio estimulo a los registros que el modelo asigno una probabilidad mayor a 0.025
* Si no envio estimulo, no gano ni pierdo, es decir la ganacia es cero
* Si el registro al que envio el estímulo es un **BAJA+2**  entonces la ganancia es de +117000
* Si el registro al que envio el estímulo es un **BAJA+1**  entonces la ganancia es de  -3000
* Si el registro al que envio el estímulo es un **CONTINUA** entonces la ganancia es de  -3000

##### A la condición que solo envio a los registros con probabilidad mayor a 1/40 la resuelvo con la condicion lógica  (prob_baja2> 0.025)
<br>
Esta condicion termina siendo un 0 o un 1
<br>
vale 1 si y solo si se cumple la condicion, es decir  si prob_baja2 es mayor a 0.025

### La ganancia de +117000 o -3000 la resuelvo de esta forma ifelse( clase_ternaria=="BAJA+2", 117000, -3000)

Finalmente,  que se cumplan ambas condiciones al mismo tiempo, en este caso, es multiplicarlas, o sea
<br>
(prob_baja2> 0.025) * ifelse( clase_ternaria=="BAJA+2", 117000, -3000)
<br>
La condicion anterior toma los siguientes valores

| prob(BAJA+2) | clase_ternaria | ganancia |
| --- | :---: | ---: |
| menor o igual a 0.025 | BAJA+1 | 0 |
| menor o igual a 0.025 | CONTINUA | 0 |
| menor o igual a 0.025 | BAJA+2 | 0 |
| mayor a 0.025 | BAJA+1 | -3000 |
| mayor a 0.025 | CONTINUA | -3000 |
| mayor a 0.025 | BAJA+2 | 117000 |


Finalmente hago el calculo en data.table

In [ ]:
ganancia_testing <- dtest[ , sum(  (prob_baja2>0.025) * ifelse( clase_ternaria=="BAJA+2", 117000, -3000) )]

In [ ]:
ganancia_testing

Es importante que la ganancia esta calculada sobre los datos de testing, que en este caso son apenas el 30%
<br>
Si quiero extrapolar a todo el dataset, debo hacer el **cociente** de esa ganancia por 0.30   ( coloquialmente, debo dividir por 0.30 )

In [ ]:
ganancia_testing_normalizada  <-  ganancia_testing/0.3

In [ ]:
ganancia_testing_normalizada